# Ejercicio 1

#### Librerias

In [24]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

1. Cargar y explorar el conjunto de datos

In [25]:
import numpy as np

# Cargar el conjunto de datos
data = np.loadtxt('./Archivos/P1_2.txt')  # Cambia el nombre del archivo según tu número de matrícula

data


array([[ 1.        ,  1.        ,  0.20471879, ..., -0.38403021,
         0.79797814,  0.89686051],
       [ 1.        ,  1.        ,  1.2522784 , ...,  0.40357021,
         0.4111649 ,  0.61222298],
       [ 1.        ,  1.        ,  0.07537733, ..., -0.55188302,
        -1.23183724, -1.14983922],
       ...,
       [ 2.        ,  1.        ,  0.23655468, ..., -0.73891858,
        -0.56288248,  0.01337943],
       [ 2.        ,  1.        , -0.6004833 , ...,  1.2057515 ,
         1.23724818,  0.80444006],
       [ 2.        ,  1.        , -0.99336043, ..., -1.08128608,
        -0.19145452,  0.17883186]])

2. Balanceo de datos (si es necesario)

In [26]:
x = data[:,1:]
y = data[:,0]
kf = StratifiedKFold(n_splits=5, shuffle = True)
clf = SVC(kernel = 'linear')
cv_y_test = []
cv_y_pred = []
for train_index, test_index in kf.split(x, y):
    x_train = x[train_index, :]
    y_train = y[train_index]
    clf.fit(x_train, y_train)
    x_test = x[test_index, :]
    y_test = y[test_index]
    y_pred = clf.predict(x_test)
    cv_y_test.append(y_test)
    cv_y_pred.append(y_pred)
print(classification_report(np.concatenate(cv_y_test), np.concatenate(cv_y_pred)))

              precision    recall  f1-score   support

         1.0       0.71      0.57      0.63       284
         2.0       0.93      0.96      0.95      1706

    accuracy                           0.91      1990
   macro avg       0.82      0.77      0.79      1990
weighted avg       0.90      0.91      0.90      1990



In [27]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear las clases
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(x, y)

# Verificar el balance
clase_1_resampled = np.sum(y_resampled == 1)
clase_2_resampled = np.sum(y_resampled == 2)
print(f'Número de muestras de clase 1 después de SMOTE: {clase_1_resampled}')
print(f'Número de muestras de clase 2 después de SMOTE: {clase_2_resampled}')


Número de muestras de clase 1 después de SMOTE: 1706
Número de muestras de clase 2 después de SMOTE: 1706


In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Crear instancias de los modelos
models = [
    LogisticRegression(solver='liblinear', max_iter=1000),
    DecisionTreeClassifier(),
    SVC(),
    KNeighborsClassifier(),
    RandomForestClassifier()
]

for model in models:
    # Realizar validación cruzada y hacer predicciones
    y_pred = cross_val_predict(model, X_resampled, y_resampled, cv=5)
    
    # Calcular métricas
    accuracy = accuracy_score(y_resampled, y_pred)
    precision = precision_score(y_resampled, y_pred)
    recall = recall_score(y_resampled, y_pred)
    f1 = f1_score(y_resampled, y_pred)
    confusion = confusion_matrix(y_resampled, y_pred)
    
    print(f'Modelo: {model.__class__.__name__}')
    print(f'Precisión: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1: {f1}')
    print(f'Matriz de Confusión:')
    print(confusion)
    print('----------------------------------------')




Modelo: LogisticRegression
Precisión: 0.8903868698710434
Recall: 0.9021101992966002
F1: 0.8916570104287369
Matriz de Confusión:
[[1539  167]
 [ 207 1499]]
----------------------------------------
Modelo: DecisionTreeClassifier
Precisión: 0.8420281359906213
Recall: 0.8792497069167644
F1: 0.8476970895733258
Matriz de Confusión:
[[1500  206]
 [ 333 1373]]
----------------------------------------
Modelo: SVC
Precisión: 0.9753810082063306
Recall: 0.9835873388042204
F1: 0.9755813953488373
Matriz de Confusión:
[[1678   28]
 [  56 1650]]
----------------------------------------
Modelo: KNeighborsClassifier
Precisión: 0.6351113716295428
Recall: 0.9982415005861665
F1: 0.7323156310470866
Matriz de Confusión:
[[1703    3]
 [1242  464]]
----------------------------------------
Modelo: RandomForestClassifier
Precisión: 0.9742086752637749
Recall: 0.9683470105509965
F1: 0.9740566037735849
Matriz de Confusión:
[[1652   54]
 [  34 1672]]
----------------------------------------


3. Implementar la regresión logística desde cero y evaluarla

In [36]:
class LogisticRegressionS:
    def __init__(self, alpha=0.0005, max_iterations=100000):
        self.alpha = alpha
        self.max_iterations = max_iterations

    def sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        self.beta = self.fit_model(X, y, self.alpha, self.max_iterations)

    def fit_model(self, X, y, alpha=0.0005, max_iterations=100000):
        npredictors = X.shape[1]
        beta = 2 * np.random.rand(npredictors) - 1.0
        it = 0

        while (np.linalg.norm(self.gradient(X, y, beta)) > 1e-4) and (it < max_iterations):
            beta = beta - alpha * self.gradient(X, y, beta)
            it = it + 1

        return beta

    def gradient(self, X, y, beta):
        xbeta = X @ beta
        c0 = (y == 0)
        c1 = (y == 1)

        exp0 = np.exp(xbeta[c0])
        l0 = (exp0 / (1 + exp0)) * X[c0, :].transpose()

        exp1 = np.exp(xbeta[c1])
        l1 = (exp1 / (1 + exp1)) * X[c1, :].transpose()

        return l0.sum(axis=1) - l1.sum(axis=1)

    def predict(self, X):
        xbeta = X @ self.beta
        tmp = 1. / (1. + np.exp(-xbeta))
        return (tmp > 0.5).astype("int32")
    
    def accuracy(self, y_true, y_pred):
        return np.mean(y_true == y_pred)


In [37]:
# Hacemos el modelo con nuestra clase
model = LogisticRegressionS(alpha=0.0005, max_iterations=100)
model.fit(X_resampled_scaled, y_resampled)

from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled_scaled, y_resampled, test_size=0.2, random_state=42)
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular y mostrar la precisión
accuracy = model.accuracy(y_test, y_pred)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")

# Imprime un informe de clasificación detallado
print(classification_report(y_test, y_pred, zero_division=1))


Precisión del modelo: 44.95%
              precision    recall  f1-score   support

         0.0       0.00      1.00      0.00         0
         1.0       0.89      0.91      0.90       339
         2.0       1.00      0.00      0.00       344

    accuracy                           0.45       683
   macro avg       0.63      0.64      0.30       683
weighted avg       0.95      0.45      0.45       683



C:\Users\luisr\AppData\Local\Temp\ipykernel_12824\2394835784.py:32: RuntimeWarning: overflow encountered in exp
  exp1 = np.exp(xbeta[c1])
C:\Users\luisr\AppData\Local\Temp\ipykernel_12824\2394835784.py:33: RuntimeWarning: invalid value encountered in divide
  l1 = (exp1 / (1 + exp1)) * X[c1, :].transpose()
C:\Users\luisr\AppData\Local\Temp\ipykernel_12824\2394835784.py:39: RuntimeWarning: overflow encountered in exp
  tmp = 1. / (1. + np.exp(-xbeta))
